# Steatosis Bayesian Network Demonstration

Lyle D. Burgoon, Ph.D.

US Army Engineer Research and Development Center

20 April 2017


## Introduction


Bayesian networks (BNs) provide a means to probabilistically model causation -- leading to odds and probabilities for predictions given certain evidence. BNs are a natural fit for the Adverse Outcome Pathway concept as well -- where causal linkages are built between key events that lead to an adverse outcome.

Here we will demonstrate several uses for BNs in the AOP framework -- we term these AOPBNs -- or AOP Bayesian Networks.

## AOP Bayesian Network for Steatosis

The AOPBN for steatosis is an expert knowledge derived model. I built the model based on the literature and my knowledge of the pathways leading to steatosis.

In [1]:
library(gRain)

#Building the Bayes Net
yn <- c("yes", "no")
nrf2 <- cptable(~ nrf2, values=c(0.5, 0.5), levels=yn)
fxr <- cptable(~ fxr | nrf2, values=c(0.95, 0.05, 0.05, 0.95), levels=yn)
shp <- cptable(~ shp | fxr, values=c(0.95, 0.05, 0.05, 0.95), levels=yn)
lxr <- cptable(~ lxr | shp, values=c(0.05, 0.95, 0.95, 0.05), levels=yn)
ppar_alpha <- cptable(~ ppara | fxr+shp+lxr, 
                      values=c(0.50, 0.50, 0.50, 0.50, 0.50, 0.50, 0.01, 0.99,
                               0.99, 0.01, 0.50, 0.50, 0.50, 0.50, 0.01, 0.99),
                      levels=yn)
hsd17b4 <- cptable(~ hsd17b4 | ppara, values=c(0.95, 0.05, 0.05, 0.95), levels=yn)
fatty_acid_beta_oxidation <- cptable(~ fatty_acid_beta_oxidation | hsd17b4,
                                     values = c(0.99, 0.01, 0.01, 0.99), levels=yn)
steatosis <-  cptable(~steatosis | cytosolic_fatty_acids + fatty_acid_beta_oxidation, 
                      values=c(0.01, 0.99, 0.01, 0.99, 0.99, 0.01, 0.99, 0.01), levels=yn)
lrh1 <- cptable(~lrh1 | shp,
                values=c(0.05, 0.95, 0.95, 0.05), levels=yn)
pi3k <- cptable(~pi3k, values=c(0.50, 0.50), levels=yn)
mtorc2 <- cptable(~mtorc2 | ir, values=c(0.99, 0.01, 0.01, 0.99), levels=yn)
ir <- cptable(~ir, values=c(0.50, 0.50), levels=yn)
akt <- cptable(~akt | pi3k+mtorc2, values=c(0.95, 0.05, 0.05, 0.95,
                                            0.95, 0.05, 0.05, 0.95),
               levels=yn)
lfabp <- cptable(~lfabp | akt+pi3k, values=c(0.95, 0.05, 0.05, 0.05, 0.50, 0.50, 0.05, 0.95),
                 levels=yn)
pparg <- cptable(~pparg | lfabp, values=c(0.95, 0.05, 0.05, 0.95), levels=yn)
fas <- cptable(~fas | lrh1+lxr+pparg, values=c(0.95, 0.05, 0.75, 0.25, 0.75, 0.25, 0.50, 0.50,
                                               0.75, 0.25, 0.50, 0.50, 0.50, 0.50, 0.01, 0.99),
               levels=yn)
mtorc1 <- cptable(~mtorc1 | akt, values=c(0.95, 0.05, 0.05, 0.95), levels=yn)
apkc <- cptable(~apkc | pi3k, values=c(0.99, 0.01, 0.01, 0.99), levels=yn)
srebp1 <- cptable(~srebp1 | mtorc1 + apkc, values=c(0.95, 0.05, 0.05, 0.95,
                                                    0.95, 0.05, 0.05, 0.95),
                  levels=yn)
scd1 <- cptable(~scd1 | srebp1, values=c(1, 0, 0, 1), levels=yn)
lipogenesis <- cptable(~lipogenesis | scd1 + fas, values=c(0.95, 0.05, 0.05, 0.95,
                                                           0.95, 0.05, 0.05, 0.95),
                       levels=yn)
cytosolic_fatty_acids <- cptable(~cytosolic_fatty_acids | lipogenesis + lfabp, 
                                 values=c(0.99, 0.01, 0.99, 0.01, 0.99, 0.01, 0.01, 0.99),
                                 levels=yn)

plist <- compileCPT(list(nrf2, fxr, shp, lxr, ppar_alpha, hsd17b4, 
                         fatty_acid_beta_oxidation, steatosis, lrh1, pi3k, 
                         mtorc2, ir, akt, lfabp, pparg, fas, mtorc1, apkc, srebp1, scd1, lipogenesis,
                         cytosolic_fatty_acids))
bn_test <- grain(plist)

NameError: name 'library' is not defined

## Scenario 1: Identifying Sufficient Key Events

In this scenario we will identify those key events that are sufficient to infer the adverse outcome. Under Rothman's idea of sufficient (reviewed and used in [[http://onlinelibrary.wiley.com/doi/10.1111/risa.12613/abstract Burgoon et al 2016]]), those key events (or simply just one key event) that are sufficient to infer the adverse outcome are the only ones that need to be measured. Key to Rothman's idea is that sufficiency could encompass a suite of key events, not just a single key event.

The way to 